In [15]:
import pickle

import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [16]:
from sklearn.pipeline import make_pipeline

In [17]:
import mlflow
import os
from dotenv import find_dotenv, load_dotenv
load_dotenv(find_dotenv())

# CHANGES EVERY TIME YOU RESTART THE EC2 INSTANCE
TRACKING_SERVER_HOST = os.environ.get("TRACKING_SERVER_HOST")
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")
mlflow.set_experiment("green-taxi-duration")

<Experiment: artifact_location='s3://agifford-mlflow-artifacts-remote/3', experiment_id='3', lifecycle_stage='active', name='green-taxi-duration', tags={}>

In [12]:
# RUN_ID = "72f8ea3dcf6548789a08b95f3a6d3375"

# # DON'T NEED TO LOAD THE DICTVECTORIZER AS IN THE TUTORIAL VIDEO BECAUSE MY MODEL IS 
# # SAVED AS A PIPELINE WITH THE DICTVECTORIZER AND RF REGRESSOR
# logged_model = f'runs:/{RUN_ID}/model'
# sk_model = mlflow.sklearn.load_model(logged_model)
# sk_model

Pipeline(steps=[('dictvectorizer', DictVectorizer()),
                ('randomforestregressor',
                 RandomForestRegressor(max_depth=20, min_samples_leaf=10,
                                       n_jobs=-1, random_state=0))])

In [4]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df


def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [6]:
df_train = read_dataframe('../../01-intro/data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('../../01-intro/data/green_tripdata_2021-02.parquet')

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

In [7]:
with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)

    pipeline = make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params, n_jobs=-1)
    )

    pipeline.fit(dict_train, y_train)
    y_pred = pipeline.predict(dict_val)

    rmse = mean_squared_error(y_pred, y_val, squared=False)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)

    mlflow.sklearn.log_model(pipeline, artifact_path="model")

{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 6.7558229919200725


/home/ubuntu/anaconda3/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [18]:
from mlflow.tracking import MlflowClient


I don't think I need the rest of this to move forward with the rest of the tutorial. My run is saved in my s3 bucket in experiment folder 3, the RUN_ID below is the correct id. `TRACKING_SERVER_HOST` will change when I return to this tutorial tomorrow

In [19]:
# MLFLOW_TRACKING_URI = f"http://{TRACKING_SERVER_HOST}:5000"
# don't need load_dotenv(find_dotenv()) when running this code on the server...
MLFLOW_TRACKING_URI = f"http://127.0.0.1:5000"
RUN_ID = "72f8ea3dcf6548789a08b95f3a6d3375"
# MLFLOW_TRACKING_URI = 'http://127.0.0.1:5000'
# RUN_ID = 'b4d3bca8aa8e46a6b8257fe4541b1136'

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [20]:
client.list_run_infos(experiment_id='1')

[<RunInfo: artifact_uri='s3://agifford-mlflow-artifacts-remote/1/fe26da65009c489491125673284d7f97/artifacts', end_time=1654114597779, experiment_id='1', lifecycle_stage='active', run_id='fe26da65009c489491125673284d7f97', run_uuid='fe26da65009c489491125673284d7f97', start_time=1654114589174, status='FINISHED', user_id='adamgifford_behavr'>]

In [12]:
path = client.download_artifacts(run_id=RUN_ID, path='dict_vectorizer.bin')

MlflowException: The following failures occurred while downloading one or more artifacts from s3://agifford-mlflow-artifacts-remote/3/72f8ea3dcf6548789a08b95f3a6d3375/artifacts: {'dict_vectorizer.bin': "ClientError('An error occurred (404) when calling the HeadObject operation: Not Found')"}

In [22]:
with open(path, 'rb') as f_out:
    dv = pickle.load(f_out)

In [23]:
dv

DictVectorizer()